<center>
    <h1> Tarea 4 / INF285 - Computación Científica </h1>
    <h2> Camilo Saldías G. </h2>
    <h2> camilo.saldias.12@sansano.usm.cl | 201223512-K </h2>
</center>

_Mayo 2017_

<div id='obj' />
# 1. Introducción

En este documento desarrollamos los enunciados de la Tarea 4 de Computación Científica, analizando distintos métodos iterativos para resolver sistemas de ecuaciones lineales y diferenciar el desempeño bajo distintos problemas.

# 2. Desarrollo

In [3]:
%matplotlib inline
import numpy as np
from scipy.linalg import solve
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.mlab as mlab
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact
from ipywidgets import widgets
import matplotlib
import matplotlib.cm as cm

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16


<h3> PALU: </h3>
<p style="font-size:16px"><b>a) </b> Dentro de la función visual, reemplazar la <code>función np.linalg.solve</code> (destacado en comentarios) con el método <b>PALU</b> para resolver el sistema de ecuaciones, para los puntos iniciales indicados más abajo, ¿Que observaciones puede destacar del comportamiento del método  y las soluciones encontradas? ¿Existen mejoras si aumenta el número de iteraciones? ¿Cuantas iteraciones se necesitan para saturar la solución?</p>

1. **(5 puntos)** $\mathbf{x}_0=\langle1,2\rangle$
2. **(10 puntos)** $\mathbf{x}_0=\langle0,2\rangle$
3. **(15 puntos)** $\mathbf{x}_0=\langle-2,-2\rangle$

In [54]:
def solve_triangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform back-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x

#permutation between rows i and j on matrix A
def row_perm(A, i, j):
    tmp = np.copy(A[i])
    A[i] = A[j]
    A[j] = tmp

def palu_decomp(A, show=False):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    
    #iterating through columns
    for j in range(N-1):
        #determine the new pivot
        p_index = np.argmax(np.abs(U[j:,j]))
        if p_index != 0:
            row_perm(P, j, j+p_index)
            row_perm(U, j, j+p_index)
            row_perm(L, j, j+p_index)
            
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j]
            
    np.fill_diagonal(L,1)
    return P,L,U

def solve_palu(A, b, show=False):
    P,L,U = palu_decomp(A, show)
    #A.x = b -> P.A.x = P.b = b'
    b = np.dot(P,b)
    # L.c = b' with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    return x

In [59]:
def visual(n_steps_newton=1,x00=0.5,x01=2):
    # Vector form of the initial guess
    x0=np.array([x00,x01])

    # The functions to be solved
    f1v=np.vectorize(lambda x,y: y-x**2)
    f2v=np.vectorize(lambda x,y: x**2+y**2-1)
    nf1f2v=np.vectorize(lambda x,y: np.linalg.norm(np.array([f1v(x,y),f2v(x,y)])))
    
    # The jacobian matrix for f1 and f2
    JM=np.vectorize(lambda x,y: np.array([[-2*x,1],[2*x,2*y]]))

    # Creating matrix to store all approximations made by the Newton's method
    xs=np.zeros((2,n_steps_newton+1))
    xs[:,0]=x0

    # Applying Newton's method n_steps_newton
    for n in np.arange(n_steps_newton):
        b=np.array([[f1v(xs[0,n],xs[1,n])],[f2v(xs[0,n],xs[1,n])]])
        # We use PALU this time
        xs[:,n+1]=xs[:,n]-solve_palu(JM(xs[0,n],xs[1,n]),b).T

    # Data for plotting
    x = np.linspace(-2.0, 2.0, 100)
    y = np.linspace(-2.0, 2.0, 100)
    [X,Y]=np.meshgrid(x,y)

    # Visualization! It is very important you learn about visualization in general.
    plt.figure(figsize=(15,7.5))
    
    plt.subplot(1, 2, 1)
    CS = plt.pcolor(X,Y,nf1f2v(X,Y))
    plt.contour(X, Y, f1v(X,Y), levels=[0], colors='k')
    plt.contour(X, Y, f2v(X,Y), levels=[0], colors='k')
    plt.colorbar(CS)
    plt.plot(xs[0,:],xs[1,:],'.r',markersize=5)
    plt.plot(xs[0,:],xs[1,:],'-r')
    plt.axis('tight')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    f1_at_x0=float(f1v(xs[0,-1],xs[1,-1]))
    f2_at_x0=float(f2v(xs[0,-1],xs[1,-1]))
    nf1f2v_at_x0=float(nf1f2v(xs[0,-1],xs[1,-1]))
    plt.title('Plotting $f_1=${:.16f},\n $f_2=${:.16f},\n $\|[f_1,f_2]\|_2=${:.16f}, \n $x_n=${:.16f}\n and $y_n=${:.16f}'.format(f1_at_x0,f2_at_x0,nf1f2v_at_x0,xs[0,-1],xs[1,-1]))
    
    plt.subplot(1, 2, 2)
    x_i=xs[0,:]
    y_i=xs[1,:]
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f1v(x_i,y_i)),'.',label='$|f_1|$',markersize=20)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f2v(x_i,y_i)),'d',label='$|f_2|$',markersize=15,alpha=0.5)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(nf1f2v(x_i,y_i)),'s',label='$\|[f_1,f_2]\|$',markersize=12,alpha=0.5)
    plt.xlabel('n (Número de iteraciones)')
    plt.grid(True)
    plt.legend(loc='best')
    
    plt.show()

interact(visual,n_steps_newton=(1,10),x00=(-2,2,0.1),x01=(-2,2,0.1))

<function __main__.visual>

Para el caso de $\mathbf{x}_0=\langle1,2\rangle$, son necesarias al menos 6 iteraciones para saturar la solución. En este caso, un aumento del número de iteraciones necesarias nos acerca más a la solución final, hasta llegar a saturar la solución.

Para el caso de $\mathbf{x}_0=\langle0,2\rangle$, el método PALU falla y no es capaz de calcular una solución, ya que implica una división por cero al momento de obtener las soluciones, independiente del número de iteraciones que se realicen.

Para el caso de $\mathbf{x}_0=\langle-2,-2\rangle$, se puede observar del segundo gráfico que el método diverge, lo que se hace más notorio a medida que se aumenta el número de iteraciones realizadas. Es posible concluir lo anterior ya que no se observa que las soluciones converjan a un valor determinado a medida que transcurren las iteraciones.

<h3> Métodos Iterativos: </h3>
<p style="font-size:16px"><b>b)</b> (1) Acople el método iterativo de <b>Gauss-Seidel</b> al visualizador (llamelo visual2), (2) incluya un cuarto parametro al widget para controlar el número de iteraciones de <b>Gauss-Seidel</b>, (3) use el algoritmo implementado para los siguientes puntos iniciales indicados más abajo. En ambos casos ¿Como afecta el número de iteraciones en la convergencia de la solución? </p>

1. **(30 puntos)** Construir visual2: (1) Acoplando Gauss-Seidel y (2) Incluyendo un cuarto parámetro al widget
2. **(10 puntos)** $\mathbf{x}_0=\langle1,2\rangle$
3. **(10 puntos)** $\mathbf{x}_0=\langle-2,-2\rangle$

In [56]:
def gauss_seidel(A, b, n_iter=50):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    #initial guess
    X[0] = np.zeros(n)
    #submatrices
    R = np.tril(A) #R=(L+D)
    U = A-R
    for i in range(1, n_iter):
        X[i] = solve_triangular(R, b.reshape(2)-np.dot(U, X[i-1]), upper=False)
    return X[i]

In [58]:
def visual2(n_steps_newton=2, n_steps_seidel=50, x00=0.5, x01=2):
    # Vector form of the initial guess
    x0=np.array([x00,x01])

    # The functions to be solved
    f1v=np.vectorize(lambda x,y: y-x**2)
    f2v=np.vectorize(lambda x,y: x**2+y**2-1)
    nf1f2v=np.vectorize(lambda x,y: np.linalg.norm(np.array([f1v(x,y),f2v(x,y)])))
    
    # The jacobian matrix for f1 and f2
    JM=np.vectorize(lambda x,y: np.array([[-2*x,1],[2*x,2*y]]))

    # Creating matrix to store all approximations made by the Newton's method
    xs=np.zeros((2,n_steps_newton+1))
    xs[:,0]=x0

    # Applying Newton's method n_steps_newton
    for n in np.arange(n_steps_newton):
        b=np.array([[f1v(xs[0,n],xs[1,n])],[f2v(xs[0,n],xs[1,n])]])
        # ... And now, we use Gauss-Seidel!
        xs[:,n+1]=xs[:,n]-gauss_seidel(JM(xs[0,n],xs[1,n]), b, n_steps_seidel).T

    # Data for plotting
    x = np.linspace(-2.0, 2.0, 100)
    y = np.linspace(-2.0, 2.0, 100)
    [X,Y]=np.meshgrid(x,y)

    # Visualization! It is very important you learn about visualization in general.
    plt.figure(figsize=(15,7.5))
    
    plt.subplot(1, 2, 1)
    CS = plt.pcolor(X,Y,nf1f2v(X,Y))
    plt.contour(X, Y, f1v(X,Y), levels=[0], colors='k')
    plt.contour(X, Y, f2v(X,Y), levels=[0], colors='k')
    plt.colorbar(CS)
    plt.plot(xs[0,:],xs[1,:],'.r',markersize=5)
    plt.plot(xs[0,:],xs[1,:],'-r')
    plt.axis('tight')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    f1_at_x0=float(f1v(xs[0,-1],xs[1,-1]))
    f2_at_x0=float(f2v(xs[0,-1],xs[1,-1]))
    nf1f2v_at_x0=float(nf1f2v(xs[0,-1],xs[1,-1]))
    plt.title('Plotting $f_1=${:.16f},\n $f_2=${:.16f},\n $\|[f_1,f_2]\|_2=${:.16f}, \n $x_n=${:.16f}\n and $y_n=${:.16f}'.format(f1_at_x0,f2_at_x0,nf1f2v_at_x0,xs[0,-1],xs[1,-1]))
    
    plt.subplot(1, 2, 2)
    x_i=xs[0,:]
    y_i=xs[1,:]
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f1v(x_i,y_i)),'.',label='$|f_1|$',markersize=20)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f2v(x_i,y_i)),'d',label='$|f_2|$',markersize=15,alpha=0.5)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(nf1f2v(x_i,y_i)),'s',label='$\|[f_1,f_2]\|$',markersize=12,alpha=0.5)
    plt.xlabel('n (Número de iteraciones)')
    plt.grid(True)
    plt.legend(loc='best')
    
    plt.show()

interact(visual2, n_steps_newton=(1,10), n_steps_seidel=(1,100), x00=(-2,2,0.1), x01=(-2,2,0.1))

<function __main__.visual2>

Para el caso de $\mathbf{x}_0=\langle1,2\rangle$, son necesarias al menos 6 iteraciones del método de Newton para saturar la solución. Lo anterior depende fuertemente del número de iteraciones de Gauss-Seidel utilizadas para calcular la solución: si por ejemplo se utilizan 50 iteraciones del método de Gauss-Seidel, entonces son necesarias al menos 7 iteraciones del método de Newton para saturar la solución. Por lo tanto, mientras más iteraciones del método de Gauss-Seidel sean utilizadas, menos iteraciones del método de Newton serán necesarias para llegar a saturar la solución.

Para el caso de $\mathbf{x}_0=\langle-2,-2\rangle$, y tal como ocurre con PALU en el caso anterior, se puede observar del segundo gráfico que el método diverge, lo que se hace más notorio a medida que se aumenta el número de iteraciones realizadas, tanto del método de Newton como el de Gauss-Seidel. Es posible concluir lo anterior ya que no se observa que las soluciones converjan a un valor determinado a medida que transcurren las iteraciones, independientemente del valor de iteraciones del método de Gauss-Seidel que se utilicen.

<p style="font-size:16px"><b>c) (20 Puntos)</b> Respecto al análisis realizado en el punto anterior, ¿Que información se desprende del segundo gráfico? ¿Que medida corresponde la curva $\|[f_1 , f_2]\|_2$ ? </p>

Como mencionamos en el punto anterior, del segundo gráfico se puede desprender información acerca de la convergencia del método a la solución final, ya que representa las soluciones obtenidas para cada iteración del método en cuestión. En este caso, la medida de la curva $\|[f_1 , f_2]\|_2$ correspondería a una medida de error de la solución, sabiendo que el resultado final del sistema de ecuaciones debiese ser cero. Por lo tanto, mientras más cercano $\|[f_1 , f_2]\|_2$ sea a cero, más exacta será la solución encontrada.

<div id='obj' />
# 3. Conclusiones
En este trabajo, y a través del desarrollo de las preguntas de la Tarea 4, se comprobó la importancia de la correcta selección de nuestros valores iniciales, dado que utilizar los valores incorrectos podría llevar a la no convergencia del método utilizado para obtener la solución. Además, y dado que se cuenta con las funciones necesarias para realizarlo, se podría realizar una comparación cuantitativa del tiempo necesario para correr cada uno de los algoritmos, lo que puede darnos un indicio de qué método es más óptimo que otro, aunque en este caso es difícil de cuantificar a simple vista a partir de los gráficos. Como última conclusión, podemos resaltar la importancia de los gráficos a la hora de analizar y entender el funcionamiento de los métodos numéricos, tanto directos como iterativos, y lo importante que resulta saber cómo realizar la visualización de los mismos en Jupyter Notebook.

<div id='obj' />
# 4. Referencias

* _Markdown_ https://help.github.com/articles/basic-writing-and-formatting-syntax/
* _numpy_ http://www.numpy.org
* _scipy_ https://www.scipy.org/scipylib/index.html
* _ipywidgets_ https://github.com/ipython/ipywidgets
 http://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Part%203%20-%20Plotting%20with%20Matplotlib.ipynb
* Cuaderno de Jupyter Notebook *05_linear_systems_of_equations*, obtenido del material de apoyo del curso.